# PS-03 Visual Search Demo
Interactive demonstration of the visual search pipeline

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from engine import read_tiff, normalize_bands, get_rgb_preview
from engine import TileGenerator, get_embedder, FAISSIndex

%matplotlib inline

## 1. Load and Visualize Sample Image

In [ ]:
# Load sample image
image_path = '../data/sample_set/GC01PS03S0001.tif'
image, metadata = read_tiff(image_path)

print(f"Image shape: {image.shape}")
print(f"Bands: {metadata['count']}")
print(f"Size: {metadata['width']}x{metadata['height']}")

In [ ]:
# Visualize RGB
image_norm = normalize_bands(image)
rgb = get_rgb_preview(image_norm)

plt.figure(figsize=(12, 8))
plt.imshow(rgb)
plt.title('Sample Image (RGB)')
plt.axis('off')
plt.show()

## 2. Generate Tiles

In [ ]:
# Create tiler
tiler = TileGenerator(tile_size=384, stride=192, scales=[1.0])
tiles = tiler.tile_image(image_norm)

print(f"Generated {len(tiles)} tiles")

In [ ]:
# Visualize first 9 tiles
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
axes = axes.flatten()

for i, tile in enumerate(tiles[:9]):
    tile_rgb = get_rgb_preview(tile.data)
    axes[i].imshow(tile_rgb)
    axes[i].set_title(f'Tile {i+1} ({tile.x}, {tile.y})')
    axes[i].axis('off')

plt.tight_layout()
plt.show()

## 3. Extract Embeddings

In [ ]:
# Load embedder
embedder = get_embedder(architecture='resnet18', device='cpu')

import torch
# Extract embeddings
tile_data = np.stack([t.data for t in tiles[:10]])
tile_tensor = torch.from_numpy(tile_data).float()

with torch.no_grad():
    embeddings = embedder(tile_tensor)
    embeddings = embeddings.numpy()

print(f"Embeddings shape: {embeddings.shape}")

## 4. Visualize Similarity

In [ ]:
# Compute similarity matrix
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(embeddings)

plt.figure(figsize=(10, 8))
plt.imshow(similarity, cmap='viridis')
plt.colorbar(label='Cosine Similarity')
plt.title('Tile Similarity Matrix')
plt.xlabel('Tile Index')
plt.ylabel('Tile Index')
plt.show()